In [1]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


/home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
forest = datasets.fetch_covtype()

In [3]:
type(forest)

sklearn.datasets.base.Bunch

In [4]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [5]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


## Maintenant un peu de xgboost!

### D'abord un tout petit

In [6]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [11]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 4
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[3]	eval-auc:0.866843	train-auc:0.867408


In [8]:
X_test[0, [0,25,35,15]]

array([ 2687.,     0.,     0.,     0.])

In [9]:
leaves = bst.predict(dtest, pred_leaf=True)
z = bst.predict(dtest)
leaves[:3], z[:3], inv_logit(0.388039+ 0.354413)

(array([[73, 77, 78, 81],
        [73, 78, 76, 88],
        [73, 77, 78, 82]], dtype=int32),
 array([ 0.75847626,  0.63132304,  0.7231403 ], dtype=float32),
 0.67753180825889836)

In [10]:
leaves[400], y[400], X_test[400, [0,7,3]]

(array([ 97, 102,  99,  89], dtype=int32),
 False,
 array([ 3126.,   254.,   283.]))

In [11]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

booster[0]:
0:[f0<3074] yes=1,no=2,missing=1
	1:[f0<2501] yes=3,no=4,missing=3
		3:[f0<2466] yes=7,no=8,missing=7
			7:[f44<0.5] yes=15,no=16,missing=15
				15:[f23<0.5] yes=31,no=32,missing=31
					31:[f46<0.5] yes=61,no=62,missing=61
						61:leaf=-0.587316
						62:leaf=0.381818
					32:[f12<0.5] yes=63,no=64,missing=63
						63:leaf=-0.537327
						64:leaf=-0.257143
				16:leaf=0.547826
			8:[f10<0.5] yes=17,no=18,missing=17
				17:[f23<0.5] yes=33,no=34,missing=33
					33:[f46<0.5] yes=65,no=66,missing=65
						65:leaf=-0.479532
						66:leaf=0.447273
					34:[f5<1358] yes=67,no=68,missing=67
						67:leaf=-0.0828358
						68:leaf=-0.573723
				18:[f9<5146] yes=35,no=36,missing=35
					35:[f5<760] yes=69,no=70,missing=69
						69:leaf=0.218182
						70:leaf=0.58209
					36:[f1<155] yes=71,no=72,missing=71
						71:leaf=-0.546667
						72:leaf=-0.0666667
		4:[f25<0.5] yes=9,no=10,missing=9
			9:[f35<0.5] yes=19,no=20,missing=19
				19:[f15<0.5] yes=37,no=38,missing=37
					37:[f36<

In [12]:
X.shape, y.shape, X_test.shape, y_test.shape

((406708, 54), (406708,), (174304, 54), (174304,))

In [16]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[40]	eval-auc:0.922714	train-auc:0.925346
[80]	eval-auc:0.944923	train-auc:0.948678
[120]	eval-auc:0.957583	train-auc:0.96204
[160]	eval-auc:0.964993	train-auc:0.969911
[200]	eval-auc:0.97121	train-auc:0.976483
[240]	eval-auc:0.975295	train-auc:0.980906
[280]	eval-auc:0.97817	train-auc:0.983926
[320]	eval-auc:0.980859	train-auc:0.986674
[360]	eval-auc:0.982833	train-auc:0.98864
[400]	eval-auc:0.98469	train-auc:0.990505
[440]	eval-auc:0.985995	train-auc:0.991836
[480]	eval-auc:0.987254	train-auc:0.993098
[520]	eval-auc:0.988187	train-auc:0.993982
[560]	eval-auc:0.989098	train-auc:0.994851
[600]	eval-auc:0.98963	train-auc:0.995409
CPU times: user 2min 51s, sys: 5.6 s, total: 2min 56s
Wall time: 44.4 s


In [17]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815251	train-auc:0.816075
[40]	eval-auc:0.9265	train-auc:0.929326
[80]	eval-auc:0.94302	train-auc:0.946318
[120]	eval-auc:0.956107	train-auc:0.960191
[160]	eval-auc:0.964489	train-auc:0.969099
[200]	eval-auc:0.970554	train-auc:0.975652
[240]	eval-auc:0.975686	train-auc:0.98084
[280]	eval-auc:0.978581	train-auc:0.983946
[320]	eval-auc:0.981304	train-auc:0.98677
[360]	eval-auc:0.983529	train-auc:0.989086
[400]	eval-auc:0.984916	train-auc:0.990508
[440]	eval-auc:0.986242	train-auc:0.991837
[480]	eval-auc:0.987173	train-auc:0.992835
[520]	eval-auc:0.988132	train-auc:0.993809
[560]	eval-auc:0.988981	train-auc:0.99464
[600]	eval-auc:0.989828	train-auc:0.995467
CPU times: user 3min 23s, sys: 1.36 s, total: 3min 24s
Wall time: 51.3 s


In [21]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.830985	train-auc:0.830948
[40]	eval-auc:0.915693	train-auc:0.918135
[80]	eval-auc:0.939123	train-auc:0.94286
[120]	eval-auc:0.951646	train-auc:0.95623
[160]	eval-auc:0.960016	train-auc:0.965135
[200]	eval-auc:0.966127	train-auc:0.971741
[240]	eval-auc:0.970492	train-auc:0.976488
[280]	eval-auc:0.973321	train-auc:0.979597
[320]	eval-auc:0.975859	train-auc:0.982351
[360]	eval-auc:0.978057	train-auc:0.984724
[400]	eval-auc:0.980107	train-auc:0.986822
[440]	eval-auc:0.981788	train-auc:0.988533
[480]	eval-auc:0.982918	train-auc:0.98977
[520]	eval-auc:0.984031	train-auc:0.99096
[560]	eval-auc:0.985204	train-auc:0.992121
[600]	eval-auc:0.986085	train-auc:0.99296
[640]	eval-auc:0.986882	train-auc:0.993774
[680]	eval-auc:0.987448	train-auc:0.994378
[720]	eval-auc:0.987992	train-auc:0.994973
[760]	eval-auc:0.988323	train-auc:0.995418
[800]	eval-auc:0.988865	train-auc:0.995881
[840]	eval-auc:0.989195	train-auc:0.996257
[880]	eval-auc:0.98952	train-auc:0.996576
[920]	eval-auc:0.9898

In [13]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 200
               )


[0]	eval-auc:0.813996	train-auc:0.814956
[200]	eval-auc:0.964482	train-auc:0.970541
[400]	eval-auc:0.979837	train-auc:0.987093
[600]	eval-auc:0.985612	train-auc:0.993219
[800]	eval-auc:0.988232	train-auc:0.996087
[1000]	eval-auc:0.989941	train-auc:0.997669
CPU times: user 25min 20s, sys: 11.9 s, total: 25min 32s
Wall time: 3min 20s


In [9]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 40
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=40,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...
Training until validation scores don't improve for 40 rounds.
[200]	valid_0's auc: 0.962666	valid_0's l2: 0.0813967
[400]	valid_0's auc: 0.978688	valid_0's l2: 0.0631855
[600]	valid_0's auc: 0.984547	valid_0's l2: 0.0544109
[800]	valid_0's auc: 0.98774	valid_0's l2: 0.0487787
[1000]	valid_0's auc: 0.989613	valid_0's l2: 0.0450177
[1200]	valid_0's auc: 0.990915	valid_0's l2: 0.0422525
[1400]	valid_0's auc: 0.991856	valid_0's l2: 0.0401496
[1600]	valid_0's auc: 0.992421	valid_0's l2: 0.0386978
[1800]	valid_0's auc: 0.992913	valid_0's l2: 0.0375098
[2000]	valid_0's auc: 0.993309	valid_0's l2: 0.036429
[2200]	valid_0's auc: 0.993696	valid_0's l2: 0.0354155
[2400]	valid_0's auc: 0.993968	valid_0's l2: 0.0345563
[2600]	valid_0's auc: 0.994214	valid_0's l2: 0.0338475
[2800]	valid_0's auc: 0.994393	valid_0's l2: 0.0332608
[3000]	valid_0's auc: 0.994585	valid_0's l2: 0.0327042
Feature names: ['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6', '

In [6]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'device': 'gpu'
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...


LightGBMError: b'GPU Tree Learner was not enabled in this build. Recompile with CMake option -DUSE_GPU=1'